#Case Study 1 : Collecting Data from Twitter

** Due Date: February 10, before the class**

*------------

**TEAM Members:**
    
    Haley Huang
    Helen Hong
    Tom Meagher
    Tyler Reese

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

#Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [73]:
import io
import json
import twitter

#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = '92TpJf8O0c9AWN3ZJjcN8cYxs'
    CONSUMER_SECRET ='dyeCqzI2w7apETbTUvPai1oCDL5oponvZhHSmYm5XZTQbeiygq'
    OAUTH_TOKEN = '106590533-SEB5EGGoyJ8EsjOKN05YuOQYu2rg5muZgMDoNrqN'
    OAUTH_TOKEN_SECRET = 'BficAky6uGyGfRzDGJqZYVKo0HS6G6Ex3ijYW3zy3kjNJ'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

#----------------------------------------------

def save_json(filename, data):
    """
    Save json data to a filename
    """
    print 'Saving data into {0}.json...'.format(filename)
    with io.open('{0}.json'.format(filename), 
                 'w', encoding='utf-8') as f:
        f.write(unicode(json.dumps(data, ensure_ascii=False)))

def load_json(filename):
    """
    Load json data from a filename
    """
    print 'Loading data from {0}.json...'.format(filename)
    with open('{0}.json'.format(filename)) as f:    
        return json.load(f)

twitter_api = oauth_login()
if twitter_api:
    print 'Bingo! API set up!'
else:
    print 'Hmmmm, something is wrong here', twitter_api

topic_of_interest = "jack"
tweets = twitter_api.statuses.user_timeline(screen_name="%s" % topic_of_interest) # SUBSTITUTE FOR STREAMING API (MIN 200 TWEETS)

# Count number of saved tweets
total_tweet_count = 0
for tweet in tweets: total_tweet_count+=1
print "%s tweets captured from topic of interest: %s." % (total_tweet_count, topic_of_interest)

# Save tweets to file
filename = "tweets"
save_json(filename, tweets)

Bingo! API set up!
19 tweets captured from topic of interest: jack.
Saving data into tweets.json...


In [74]:
# Load saved tweets into results
results = load_json(filename)

Loading json from tweets.json...


In [64]:
# Compute Additional Statistics about the tweets collected

# Determine the average number of words in the text of each tweet
def average_words(tweet_texts):
    total_words =  sum([len(s.split()) for s in tweet_texts])
    return 1.0*total_words/len(tweet_texts)

tweet_texts = [ tweet['text'] 
                 for tweet in results ]

print 'Average number of words:', average_words(tweet_texts)

# Calculate the lexical diversity of all words contained in the tweets
def lexical_diversity(tokens):
    return 1.0*len(set(tokens))/len(tokens)

words = [ word 
          for tweet in tweet_texts 
              for word in tweet.split() ]

print 'Lexical Diversity:', lexical_diversity(words)

Average number of words: 14.2631578947
Lexical Diversity: 0.789667896679


###Report some statistics about the tweets you collected 

*The topic of interest: jack

*The total number of tweets collected:  19

*-----------------------

#Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [128]:
from collections import Counter
from prettytable import PrettyTable

tweet_texts = [ tweet['text'] 
                 for tweet in results ]
words = [ word 
          for tweet in tweet_texts 
              for word in tweet.split()
                 if word.lower() not in ['rt', 'the'] # FILTER OUT STOP WORDS (RT, THE)
        ]

# frequency of words
count = Counter(words).most_common()

# table of the top 30 words with their counts
pretty_table = PrettyTable(field_names=['Word', 'Count']) 
[ pretty_table.add_row(w) for w in count[:30] ]
pretty_table.align['Word'] = 'l'
pretty_table.align['Count'] = 'r'
print pretty_table

+-------------+-------+
| Word        | Count |
+-------------+-------+
| to          |     6 |
| your        |     4 |
| on          |     4 |
| this        |     4 |
| our         |     3 |
| out         |     3 |
| for         |     3 |
| new         |     3 |
| of          |     3 |
| way         |     3 |
| my          |     3 |
| is          |     3 |
| in          |     3 |
| just        |     2 |
| got         |     2 |
| small       |     2 |
| #IowaCaucus |     2 |
| @Square     |     2 |
| reader      |     2 |
| across      |     2 |
| only        |     2 |
| more        |     2 |
| with        |     2 |
| you         |     2 |
| a           |     2 |
| Apple       |     2 |
| https://…   |     1 |
| @kmac_TX:   |     1 |
| show        |     1 |
| @adambain:  |     1 |
+-------------+-------+


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [56]:
from collections import Counter
from prettytable import PrettyTable

# Create a list of all tweets with at least one retweet, and index the originator of that tweet and the text.
retweets = [
            (tweet['retweet_count'],
            tweet['retweeted_status']['user']['screen_name'],
            tweet['text'])
            
            #Ensure that a retweet exists
            for tweet in results                      
                if tweet.has_key('retweeted_status')
            ]

pretty_table = PrettyTable(field_names = ['Count','Screen Name','Text'])

# Sort tweets by descending number of retweets and display the top 10 results in a table.
[pretty_table.add_row(row) for row in sorted(retweets, reverse = True)[:10]]
pretty_table.max_width['Text'] = 50
pretty_table.align = 'l'
print pretty_table

+-------+--------------+----------------------------------------------------+
| Count | Screen Name  | Text                                               |
+-------+--------------+----------------------------------------------------+
| 1089  | JohnMayer    | RT @JohnMayer: LIVE on #Periscope: '52 Tele Blues  |
|       |              | https://t.co/ktnmFiJLSg                            |
| 342   | Support      | RT @Support: New on https://t.co/zDdcbPvEwm:       |
|       |              | entire Tweet conversations pop out with a single   |
|       |              | click––no more losing your place! https://t.c…     |
| 174   | Mospeights16 | RT @Mospeights16: 😂😂😂RT @ThirtySecSport: NBA    |
|       |              | players reading mean tweets 😂😂                   |
|       |              | https://t.co/spJg2Rpq7R                            |
| 160   | gomery       | RT @gomery: Currently in parts of Iowa.            |
|       |              | #IowaCaucus  https://t.co/QRNo0LznxE        

Another measure of tweet "popularity" could be the number of times it is favorited.  The following calculates the top-10 tweets with the most "favorites"

In [57]:
from prettytable import PrettyTable

# Determine the number of "favorites" for each tweet collected.

favorites = [
            (tweet['favorite_count'],
             tweet['text'])
            for tweet in results
            ]
            
pretty_table = PrettyTable(field_names = ['Count','Text'])

# Sort tweets by descending number of favorites and display the top 10 results in a table.
[pretty_table.add_row(row) for row in sorted(favorites, reverse = True)[:10]]
pretty_table.max_width['Text'] = 75
pretty_table.align = 'l'
print pretty_table

+-------+-----------------------------------------------------------------------------+
| Count | Text                                                                        |
+-------+-----------------------------------------------------------------------------+
| 215   | "…the only way to make real change is to go out of your way to do so."      |
|       | https://t.co/VPNBn0AJ8E                                                     |
| 208   | "Twitter nails it." #IowaCaucus https://t.co/yMaJPOlJNZ                     |
| 206   | 💯💯💯 https://t.co/vU0ne0wEHK                                              |
| 143   | This is a fascinating way to experience an election year. Right on the      |
|       | ground live where it's unfolding. https://t.co/DgqMGpbbzt                   |
| 74    | Broadcasts from #GroundhogDay this morning: https://t.co/H0xPLQiyuU         |
| 53    | Amazing  https://t.co/OzFSGU2avc                                            |
| 13    | @ThePayam @Square thank y

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [66]:
from collections import Counter
from prettytable import PrettyTable

# Extract the screen names which appear among the collection of tweets
screen_names = [user_mention['screen_name']
               for tweet in results
                   for user_mention in tweet['entities']['user_mentions']]

# Extract the hashtags which appear among the collection of tweets
hashtags = [ hashtag['text']
           for tweet in results
               for hashtag in tweet['entities']['hashtags']]

# Simultaneously determine the frequency of screen names/hashtags, and display the top 10 most common in a table.
for label, data in (('Screen Name',screen_names),
                   ('Hashtag',hashtags)):
    pretty_table = PrettyTable(field_names =[label,'Count'])
    counter = Counter(data)
    [ pretty_table.add_row(entity) for entity in counter.most_common()[:10]]
    pretty_table.align[label] ='l'
    pretty_table.align['Count'] = 'r'
    print pretty_table


+----------------+-------+
| Screen Name    | Count |
+----------------+-------+
| Square         |     4 |
| ThePayam       |     2 |
| TryCaviar      |     1 |
| patrickc       |     1 |
| NickPacilio    |     1 |
| Mospeights16   |     1 |
| JohnMayer      |     1 |
| gomery         |     1 |
| kmac_TX        |     1 |
| ThirtySecSport |     1 |
+----------------+-------+
+---------------+-------+
| Hashtag       | Count |
+---------------+-------+
| IowaCaucus    |     2 |
| KeepPounding  |     1 |
| Broncos       |     1 |
| SuperBowl     |     1 |
| GroundhogDay  |     1 |
| CaviarGameDay |     1 |
| GameDay       |     1 |
| Periscope     |     1 |
| SB50          |     1 |
| Panthers      |     1 |
+---------------+-------+


*------------------------

#Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [67]:
#----------------------------------------------
import sys
import time
from urllib2 import URLError
from httplib import BadStatusLine
import json
from functools import partial
from sys import maxint

# The following is the "general-purpose API wrapper" presented in "Mining the Social Web" for making robust twitter requests.
# This function can be used to accompany any twitter API function.  It force-breaks after receiving more than max_errors
# error messages from the Twitter API.
def make_twitter_request(twitter_api_func, max_errors = 10, *args, **kw):
    def handle_twitter_http_error(e, wait_period = 2, sleep_when_rate_limited = True):
        
        if wait_period > 3600:
            print >> sys.stderr, 'Too many retries. Quitting.'
            raise e
        
        if e.e.code == 401:
            print >> sys.stderr, 'Encountered 401 Error (Not Authorized)'
            return None
        
        elif e.e.code == 404:
            print >> sys.stderr, 'Encountered 404 Error (Not Found)'
            return None
        
        elif e.e.code == 429:
            print >> sys.stderr, 'Encountered 429 Error (Rate Limit Exceeded)'
            if sleep_when_rate_limited:
                print >> sys.stderr, "Retrying again in 15 Minutes...ZzZ..."
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print >> sys.stderr, '...ZzZ...Awake now and trying again.'
                return 2
            else:
                raise e
                
        elif e.e.code in (500,502,503,504):
            print >> sys.stderr, 'Encountered %i Error.  Retrying in %i seconds' % \
                (e.e.code, wait_period)
            time.sleel(wait.period)
            wait.period *= 1.5
            return wait_period
        
        else:
            raise e
            
    wait_period = 2
    error_count = 0
    
    while True:
        try:
            return twitter_api_func(*args,**kw)
        except twitter.api.TwitterHTTPError, e:
            error_count = 0
            wait_period = handle_twitter_http_error (e, wait_period)
            if wait_period is None:
                return
            
        except URLError, e:
            error_count += 1
            print >> sys.stderr, "URLError encountered.  Continuing"
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise
        
        except BadStatusLine, e:
            error_count += 1
            print >> sys.stderr, "BadStatusLineEncountered.  Continuing"
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise

# Get Twitter friends and followers from "Mining the Social Web".
def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                              friends_limit=maxint, followers_limit=maxint):
    
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    # See https://dev.twitter.com/docs/api/1.1/get/friends/ids and
    # https://dev.twitter.com/docs/api/1.1/get/followers/ids for details
    # on API parameters
    
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids, 
                              count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids, 
                                count=5000)

    friends_ids, followers_ids = [], []
    
    for twitter_api_func, limit, ids, label in [
                    [get_friends_ids, friends_limit, friends_ids, "friends"], 
                    [get_followers_ids, followers_limit, followers_ids, "followers"]
                ]:
        
        if limit == 0: continue
        
        cursor = -1
        while cursor != 0:
        
            # Use make_twitter_request via the partially bound callable...
            if screen_name: 
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
        
            print 'Fetched {0} total {1} ids for {2}'.format(len(ids), 
                                                    label, (user_id or screen_name))
        
            # XXX: You may want to store data during each iteration to provide an 
            # an additional layer of protection from exceptional circumstances
        
            if len(ids) >= limit or response is None:
                break

    # Do something useful with the IDs, like store them to disk...
    return friends_ids[:friends_limit], followers_ids[:followers_limit]
                
# Get Twitter user profiles from "Mining the Social Web".
def get_user_profile(twitter_api, screen_names=None, user_ids=None):
   
    # Must have either screen_name or user_id (logical xor)
    assert (screen_names != None) != (user_ids != None), \
    "Must have screen_names or user_ids, but not both"
    
    items_to_info = {}

    items = screen_names or user_ids
    
    while len(items) > 0:

        # Process 100 items at a time per the API specifications for /users/lookup.
        # See https://dev.twitter.com/docs/api/1.1/get/users/lookup for details.
        
        items_str = ','.join([str(item) for item in items[:100]])
        items = items[100:]

        if screen_names:
            response = make_twitter_request(twitter_api.users.lookup, 
                                            screen_name=items_str)
        else: # user_ids
            response = make_twitter_request(twitter_api.users.lookup, 
                                            user_id=items_str)
    
        for user_info in response:
            if screen_names:
                items_to_info[user_info['screen_name']] = user_info
            else: # user_ids
                items_to_info[user_info['id']] = user_info

    return items_to_info

In [131]:
from collections import Counter
from prettytable import PrettyTable

# friends_ids, followers_ids = get_friends_followers_ids(twitter_api, 
#                                                        screen_name="jack", 
#                                                        friends_limit=None, 
#                                                        followers_limit=None)

# friends_profiles = get_user_profile(twitter_api, user_ids=friends_ids)
# followers_profiles = get_user_profile(twitter_api, user_ids=followers_ids)
# save_json('friends', friends_profiles)
# save_json('followers',followers_profiles)

friends_json = load_json('friends')
followers_json = load_json('followers')

# Extract the ids and screen names for friends
friends_ids_and_screen_names = [(user_id, friends_json[user_id]['screen_name']) for user_id in friends_json]
    
# Extract the ids and screen names for followers
followers_ids_and_screen_names = [(user_id, followers_json[user_id]['screen_name']) for user_id in followers_json]

# Display the 20 followers and friends for user
for label, data in (('Followers',followers_ids_and_screen_names),
                   ('Friends',friends_ids_and_screen_names)):
    pretty_table = PrettyTable(field_names =['Id','Screen Name'])
    [ pretty_table.add_row(user) for user in data[:20]]
    pretty_table.align['Id'] ='l'
    pretty_table.align['Screen Name'] = 'l'
    print pretty_table

Loading json from friends.json...
Loading json from followers.json...
+--------------------+-----------------+
| Id                 | Screen Name     |
+--------------------+-----------------+
| 4871056991         | sebasti44989788 |
| 14180315           | gravenerito     |
| 694607378638045185 | YuzerMehdi      |
| 4871797325         | qiP4wvbeNkdm97B |
| 4873934668         | vaLeRiya67889   |
| 4871611353         | 1907talha67     |
| 4872587775         | buraqqtr        |
| 4872883319         | artyharpist     |
| 116615520          | lisamitchell_rf |
| 4847832072         | sam_yankun      |
| 4846040714         | 769_tweetsalot  |
| 4871796693         | btimperfection  |
| 4685551056         | gezere49        |
| 4870572869         | zunairach9      |
| 454727757          | hayles___       |
| 106951183          | BobIntoccia     |
| 4870817733         | laurencemendez3 |
| 30512024           | Bpinches        |
| 4874086438         | dD6ukQVagjy0KfO |
| 4872146967         | minys

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [130]:
# This function views the friend and follower lists computed above as mathematical sets.  It then computes the set-intersection 
# to determine mutual friends.  It also uses set-differences to determine unbalanced friendships.  It returns to the user:
    # Number of friends
    # Number of followers
    # Number of mutual friends
    # Number of friens which are not followers
    # Number of followers which are not friends
    
def setwise_friends_followers_analysis(screen_name, friends_ids, followers_ids):
    friends_ids, followers_ids = set(friends_ids),set(followers_ids)
    
    print '{0} is following {1}'.format(screen_name, len(friends_ids))
    
    print '{0} is being followed by {1}'.format(screen_name,len(followers_ids))
    print '{0} has {1} mutual friends'.format(screen_name, len(friends_ids.intersection(followers_ids)))
    print'{0} of {1} are not following {2} back'.format(len(friends_ids.difference(followers_ids)),len(friends_ids),screen_name)
    print '{0} of {1} are not being followed back by {2}'.format(len(followers_ids.difference(friends_ids)),len(followers_ids),screen_name)


setwise_friends_followers_analysis('User',friends_ids,followers_ids)

User is following 1813
User is being followed by 5000
User has 0 mutual friends
1813 of 1813 are not following User back
5000 of 5000 are not being followed back by User


*------------------------

#Problem 4: Explore the data 

Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API

In [2]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through myWPI, in the Assignment "Case Study 1".
        
** Note: Each team just need to submit one submission in myWPI **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Explore the data
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

